# Export to GeoJSON or File GeoDatabase with Indicator Attributes

This Jupyter Notebook will walk you through the process of converting your DevInfo database into the possible output formats of:
- GeoJSON (only spatial information and a reference area id)
- GeoJSON (full attribute AND spatial information)
- FileGeodatabase

### **Important**

You need to have previously run the Export to Shapefiles notebook **before** running this notebook as it will need a folder location of the individual shapefiles. 

Since we are using the **[ArcPy](http://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm)** python library, we will need to start our Jupyter Notebook the following way:

- Open ArcGIS Pro
- Click the **Project** tab and click **Python** to access the **Python Package Manager**. To create, edit or remove environments click on the **Manage Environments** button.
- Clone the existing default python environment as the default is read-only. *[reference](http://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-conda.htm#ESRI_SECTION2_1B154688AEE0497F836B3FFBAAAD0C8C)*
- Find the `pyodbc` package in the list and install it. *[reference](http://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-conda.htm#ESRI_SECTION2_85BC919097434B3B9AE1A746D793AA29)*
- **IMPORTANT** open the `Python Command Prompt` and verify the path is to your new, cloned enviornment
- use the `cd <path>` command to change to your working directory
- start your Jupyter Notebook server using the `jupyter notebook` command

## Import the needed python libraries

In [ ]:
import os
import json
import pyodbc
import arcpy

## Set defaults

In [ ]:
# location of access database
access_database = r'Z:\dev\nepal-import\NepalInfo2016_for_python.accdb'

# where is your working folder? usually just the name of the country you are working on
output_base = 'nepal'

# location of individual shapefiles for each layer. 
# this is created by running the Export to Shapefiles jupyter notebook, see *Important* note above
output_shapes_folder = 'shapes'

# where do you want to write the output geojson files?
output_geojson_folder = 'geojson_full'

# we will need to create a fgdb anyway. 
# TODO: add a clean up to keep it or delete it based on this flag
keep_fgdb = False

## Create FileGeodatabase

In [ ]:
spatial_out = 'output_fgdb'
arcpy.CreateFileGDB_management(os.path.join(output_base, spatial_out), output_base)

## Copy the template feature class to our working area
A template feature class will have the base fields we want all the new features to have

In [ ]:
# copy fc template into new fgdb
in_features = os.path.join('devinfo_templates.gdb', 'devinfo_fc_template')
out_path = os.path.join(output_base, spatial_out, '{}.gdb'.format(output_base))
out_template_name = '{}_fc_template'.format(output_base)

template_path = os.path.join(out_path, out_template_name)

Create the template feature class in the working area if it doesn't already exist

In [ ]:
if arcpy.Exists(template_path):
    print ('template feature class already exists at :: {}'.format(template_path))
else:
    arcpy.FeatureClassToFeatureClass_conversion(in_features=in_features, out_path=out_path, out_name=out_template_name)
    print ('succesfully created template feature class with fields :: {}'.format(template_path))

## Query the DevInfo Access Database

In [ ]:
connStr = (
    r'Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};'
    r'DBQ={};'.format(access_database)
)

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT 
    Indicator_NId, 
    Indicator_Name,
    Data_Value,
    Unit_NId, 
    Unit_Name,
    Subgroup_Val,
    Subgroup_Type_Name,
    TimePeriod,
    Area_Level_Name,
    Area_Level,
    Area_ID,
    Area_Name,
    Layer_NId,
    IC_Name,
    Publisher
FROM ut_data_query 
ORDER BY indicator_nid ASC
"""
crsr = cnxn.execute(sql)

rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

## Chunk up the data
Chunk up the data by Indicator. This will let us create one layer per Indicator

In [ ]:
chunks = {}
for row in rows:
    ind_id = row.Indicator_NId
    if ind_id not in chunks:
        chunks[ind_id] = {}
        chunks[ind_id]['rows'] = []
        
    chunks[ind_id]['rows'].append(row)

for c in chunks:
    print (c, len(chunks[c]['rows']))

## Create the layers and insert the data
For each Indicator, we want to create a new layer and add in *both* the attribute and the spatial data

In [ ]:
template_fields = [ 'SHAPE@', 'Indicator_NId', 'Indicator_Name', 'Data_Value', 'Unit_NId', 'Unit_Name', 'Subgroup_Val', 'Subgroup_Type_Name', 'TimePeriod', 'Area_Level_Name', 'Area_Level', 'Area_ID', 'Area_Name', 'Layer_NId', 'IC_Name', 'Publisher']

# store the already reference geometry in memory for faster recall
geom_cache = {}

for c in chunks:
    
    # test with just one
    # remove this if statement if you want to process all the indicators
    if c != 21.0:
        continue
    
    # for each indicator's "chunk" of rows..
    rows = chunks[c]['rows']
    for row in rows:
        layer_id = row.Layer_NId
        area_id = row.Area_ID

        # look for the already created shapefile
        shp_file_path = '{}/{}.shp'.format(os.path.join(output_base, output_shapes_folder), layer_id)

        # check to see if we were able to get the shapefile
        # TODO: add logging rather than just printing an exception
        try:
            desc = arcpy.Describe(shp_file_path)
        except:
            #print ('{} shape file not found'.format(shp_file_path))
            continue

        # create the layer name from the values in the row
        indicator_id = row.Indicator_NId
        new_ind = str(indicator_id)
        new_ind_id = new_ind.replace('.', '_')
        layer_name = 'indicator_{}_layer_{}'.format(new_ind_id, layer_id)
        
        # if the layer does not exist already, create it
        full_path = os.path.join(out_path, layer_name)
        if not arcpy.Exists(full_path):
            arcpy.CreateFeatureclass_management(out_path, layer_name, 'POLYGON', template_path)
            
        # check the geometry cache to see if we can just grab it from memory
        # if not, open up the shapefile and get the geometry
        geom = None
        if area_id in geom_cache:
            geom = geom_cache[area_id]
        else:
            where_clause = '{} = \'{}\''.format(area_id_field, area_id)   
            with arcpy.da.SearchCursor(shp_file_path, ['SHAPE@'], where_clause) as cursor:
                for area_rows in cursor:
                    geom = area_rows[0]
                    geom_cache[area_id] = geom

        # create the insert cursor used to actually insert the data
        ic = arcpy.da.InsertCursor(full_path, template_fields)
        
        # convert the row object to a list
        vals = list(row)
        
        # put the geometry value in the first position of the list
        vals.insert(0, geom)
                
        # insert the row
        ic.insertRow(vals)
        
        # clean up the insert cursor
        del ic
                
# delete the geometry cache from memory
del geom_cache
print ('done creating')

## Export the FileGeodatabase to GeoJSON

_exporting from a file geodatabse to geojson using the arcpy.FeaturesToJSON conversion throws a generic error. for now, manually create the geojson_

In [ ]:
# do you want all the fields or only the area id field
include_all_fields = False

# do you want to export the geometry 
include_geometry = True

# tell arcpy where to look for the feature classes by setting the workspace
arcpy.env.workspace = out_path

# loop through each feature class
for fc in arcpy.ListFeatureClasses():

    # skip the template fc
    if fc == out_template_name:
        continue

    # set the name of the output geojson file
    out_geojson = '{}/{}.geojson'.format(os.path.join(output_base, output_geojson_folder), fc)

    # Here is where arcpy throws a generic 999999 error. leave for reference and follow up
    # arcpy.FeaturesToJSON_conversion(in_features=out_loc, out_json_file=out_geojson)

    # base geojson template
    template = {
        "type": "FeatureCollection",
        "features": []
    }

    print ('collecting geojson for {}'.format(out_geojson))

    # loop through each feature and add it to the geojson template
    with arcpy.da.SearchCursor(fc, template_fields) as cursor:
        
        # get a reference to the fields in the feature class
        c_fields = cursor.fields
        
        # for reach row
        for row in cursor:
            
            # get the geometry object
            geom = row[c_fields.index('SHAPE@')]

            #setup the base geojson feature
            feature = {
                "type": "Feature",
                "properties": {}
            }
            
            # add geometry if specified
            if include_geometry:
                feature['geometry'] = geom.__geo_interface__

            # add fields if specified
            if include_all_fields:
                for f in c_fields:
                    # skip these fields in the output
                    if f in ['SHAPE@', 'OBJECTID']:
                        continue

                    feature['properties'][f] = row[c_fields.index(f)]
            else:
                feature['properties']['REF_AREA_ID'] = row[c_fields.index('Area_ID')]

            # add feature to base
            template['features'].append(feature)

    # create the geojson file on disk and write the data to it
    with open(out_geojson, 'w') as file:
        json.dump(template, file)

    print ('succesfully wrote {}'.format(out_geojson))

print ('done!')